# Behavioral Design Patterns


Behavioral patterns don't really follow any central theme:

- They are all different.
- Sometimes they overlap in their function, i.e., the goal they achieve, but the underlying mechanisms are different.

Table of contents:

- [Chain of Responsibility](#chain-of-responsibility)
  - Example: Support
  - Example: Card Game
  - Example: Broker and Queries
- [Command](#Command)
- [Interpreter](#Interpreter)
- [Iterator](#Iterator)
- [Mediator](#Mediator)
- [Memento](#Memento)
- [Observer](#Observer)
- [State](#State)
- [Strategy](#Strategy)
- [Template](#Template)
- [Visitor](#Visitor)

## Chain of Responsibility

Software components are often built with hierarchical relationships. Those relationships can capture resposibility, similarly as it can happen in a company (CEO, manager, employee). The pattern Chain of Responsibility is a chain of components in which all get a change to process a command or a query, optionally having a default processing implementation and an ability to terminate the processing chain.

In other words, Chain of Responsibility allows an object to pass a request along a chain of potential handlers until the request is handled. This pattern decouples the sender of the request from the receiver by giving multiple objects a chance to handle the request.
 
For example:

- In a GUI, we click a graphical element on a form
  - The button handles the action, which leads to stopping the processing
  - Or the underlying group box can handle it
  - Or the underlying window can handle it
- A card computer game
  - A creature has attack and defense skills
  - Thos skills can be boosted by other card creatures
- Support
  - We can have first-level or basic support
  - Or second-level = advanced support
  - Or third-level = expert support

Some key components that often appear in Chain of Responsibility are:

- **Handler**: Defines an interface for handling requests and for setting the next handler in the chain.
- **ConcreteHandler**: Implements the Handler interface and processes requests it is responsible for. If it can't handle the request, it forwards the request to the next handler in the chain.
- **Client**: Initiates the request and passes it to the first handler in the chain.

**Command and Query Separation (CQS)**: Whenever we operate on objects, we separate all the invokations into two types:

- Command: invokation which asks for an action or a change; e.g., set value to X.
- Query: ask/retrieve information without changing anything, e.g., get value of X.

Separating both Commands and Queries is known as the Command and Query Separation (CQS) Principle. It is a design priciple with some benefits:

- Clarity: Separating commands and queries makes it clear which methods are performing actions and which are retrieving information.
- Simpler Testing: Queries are easier to test because they are idempotent and do not change the state.
- Improved Maintainability: Clear separation of responsibilities makes the code easier to understand and maintain.

### Example: Support Chain

In [7]:
# Abstract Handler:
# Defines an interface for handling requests 
# and for setting the next handler in the chain.
# SupportHandler will be inherited by concrete handlers!
class SupportHandler:
    def __init__(self):
        # The next_handler is a pointer to the next handler
        # which is set with set_next_handler
        # when setting the Chain of Responsibility
        self.next_handler = None

    # This method is used to set the
    # Chain of Responsibility
    def set_next_handler(self, handler):
        self.next_handler = handler

    # This handle_request method is called by the concrete
    # class/object if the type of issue to handle 
    # does not belong to the type of the concrete handler
    def handle_request(self, issue):
        if self.next_handler:
            return self.next_handler.handle_request(issue)
        else:
            return "No handler available for this issue"

In [8]:
# Concrete Handlers:
# They implement the Handler interface 
# and processes requests it is responsible for. 
# If they can't handle the request, 
# they forward the request to the next handler in the chain.
class BasicSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "basic":
            return "BasicSupport: Handling basic issue"
        else:
            # We call the abstract handler
            # which calls the next handler
            # in the Chain of Responsibility
            return super().handle_request(issue)

class AdvancedSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "advanced":
            return "AdvancedSupport: Handling advanced issue"
        else:
            return super().handle_request(issue)

class ExpertSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "expert":
            return "ExpertSupport: Handling expert issue"
        else:
            return super().handle_request(issue)


In [9]:
# __main__: Client
# Initiates the request and passes it 
# to the first handler in the chain.

# Create handlers
basic = BasicSupport()
advanced = AdvancedSupport()
expert = ExpertSupport()

# Set up Chain of Responsibility
# Basic -> Advanced -> Expert
basic.set_next_handler(advanced)
advanced.set_next_handler(expert)

# Client issues
# In this simple example, 
# each issue is a string that specifies the type of issue,
# i.e., we use it to decide which handler should process it
issues = ["basic", "advanced", "expert", "unknown"]

for issue in issues:
    print(f"Issue: {issue} - Response: {basic.handle_request(issue)}")
# Issue: basic - Response: BasicSupport: Handling basic issue
# Issue: advanced - Response: AdvancedSupport: Handling advanced issue
# Issue: expert - Response: ExpertSupport: Handling expert issue
# Issue: unknown - Response: No handler available for this issue


Issue: basic - Response: BasicSupport: Handling basic issue
Issue: advanced - Response: AdvancedSupport: Handling advanced issue
Issue: expert - Response: ExpertSupport: Handling expert issue
Issue: unknown - Response: No handler available for this issue


### Example: Card Chain

Example with card computer game. This example is similar but different to the previous one:

- In the Support example we took and issue and passed it to the Chain of Responsibility until the correct concrete handler is found.
- In this Card Game example, we create a chain of Creature modifiers, i.e., abilities, contained in a root modifier. Then, when we want to use the root, all modifiers are used and each calls to the next in the chain, except when a specific modifier with that function prevents it from propagating.

In [1]:
# Our Card-Creature
class Creature:
    def __init__(self, name, attack, defense):
        self.defense = defense
        self.attack = attack
        self.name = name

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'

# A Creature-Card can be modified: 
# it collects/looses skills or objects: sword, poison, etc.
# This is where we define our Chain of Responsibility,
# it is our Abstract Modifier.
# We will inherit this base class to Concrete Modifiers.
class CreatureModifier:
    def __init__(self, creature):
        self.creature = creature
        self.next_modifier = None

    def add_modifier(self, modifier):
        if self.next_modifier:
            self.next_modifier.add_modifier(modifier)
        else:
            self.next_modifier = modifier

    # Here is where we apply the Chain of Responsibility
    # This handle() is called from the derived class objects
    def handle(self):
        if self.next_modifier:
            self.next_modifier.handle()

# Concrete Modifier
# Here, we define the concrete handle() methods
class DoubleAttackModifier(CreatureModifier):
    def handle(self):
        print(f'Doubling {self.creature.name}''s attack')
        self.creature.attack *= 2
        super().handle()


class IncreaseDefenseModifier(CreatureModifier):
    def handle(self):
        if self.creature.attack <= 2:
            print(f'Increasing {self.creature.name}''s defense')
            self.creature.defense += 1
        super().handle()

    
class NoBonusesModifier(CreatureModifier):
    def handle(self):
        print('No bonuses for you!')
        # Note that no super().handle() is called
        # That means we don't keep applying the Chain of Responsibility

In [10]:
# __main__
goblin = Creature('Goblin', 1, 1)
print(goblin)

# Abstract Handler: the root
root = CreatureModifier(goblin)

# Chain of Responsibility
root.add_modifier(NoBonusesModifier(goblin))
root.add_modifier(DoubleAttackModifier(goblin))
root.add_modifier(DoubleAttackModifier(goblin))

# Add another modifier - no effect
root.add_modifier(IncreaseDefenseModifier(goblin))

root.handle()  # apply modifiers
print(goblin)

Goblin (1/1)
No bonuses for you!
Goblin (1/1)


### Example: Broker and Queries

In [12]:
from abc import ABC
from enum import Enum

# List of functions we can call
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)


class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2


class Query:
    def __init__(self, creature_name, what_to_query, default_value):
        self.value = default_value  # bidirectional
        self.what_to_query = what_to_query
        self.creature_name = creature_name

# Broker
class Game:
    def __init__(self):
        self.queries = Event()

    def perform_query(self, sender, query):
        self.queries(sender, query)


class Creature:
    def __init__(self, game, name, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.name = name
        self.game = game

    @property
    def attack(self):
        q = Query(self.name, WhatToQuery.ATTACK, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.name, WhatToQuery.DEFENSE, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'


class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.creature = creature
        self.game = game
        self.game.queries.append(self.handle)

    def handle(self, sender, query):
        pass

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.game.queries.remove(self.handle)


class DoubleAttackModifier(CreatureModifier):

    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.ATTACK):
            query.value *= 2


class IncreaseDefenseModifier(CreatureModifier):

    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.DEFENSE):
            query.value += 3


In [13]:
# __main__
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)

with DoubleAttackModifier(game, goblin):
    print(goblin)
    with IncreaseDefenseModifier(game, goblin):
        print(goblin)

print(goblin)

Strong Goblin (2/2)
Strong Goblin (4/2)
Strong Goblin (4/5)
Strong Goblin (2/2)
